In [1]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import GPUtil

import src.shared.util as U
#from src.decoder import Decoder
from src.dataset.decode_dataset import DecodeDataset
from src.shared.subdivide import Subdivide

torch.set_printoptions(sci_mode=False, precision=5)

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from einops import rearrange

class Head(nn.Module):    

    def __init__(self, p_sz, emb_sz, mid_sz):
        super().__init__()
        self.p_sz = p_sz
        self.proj_emb = nn.Linear(emb_sz, mid_sz, bias=False)
        self.proj_key = nn.Linear(3*p_sz**2, mid_sz, bias=False)        
        self.net = nn.Sequential(
            nn.Linear(2*mid_sz, 2*mid_sz, bias=False),            
            nn.ReLU(True),
            nn.Linear(2*mid_sz, 3*p_sz**2, bias=False),
        ) 

    def forward(self, x, emb):        
        #emb = self.proj_emb(emb[:, None])
        emb = self.proj_emb(emb)
        u, v = x.size(-2) // self.p_sz, x.size(-1) // self.p_sz
        y = rearrange(x, 'b c (u m) (v n) -> b (u v) (c m n)',
                      m=self.p_sz, n=self.p_sz)        
        y = self.proj_key(y)        
        emb = emb.expand(-1, y.size(1), -1)
        y = y.expand(emb.size(0), -1, -1)        
        y = torch.cat([emb, y], dim=-1)        
        y = self.net(y)
        return rearrange(y, 'b (u v) (c m n) -> b c (u m) (v n)',
                         c=3, u=u, v=v, m=self.p_sz, n=self.p_sz)
        

class MultiHead(nn.Module):
    def __init__(self, p_sz, e_sz, mid_sz, sizes):
        super().__init__()
        self.p_sz = p_sz
        self.sizes = sizes
        self.heads = nn.ModuleList([
            Head(p_sz, e_sz, mid_sz) 
            for _ in sizes])

    def forward(self, emb, mean):        
        layers = [mean]
        for i, (head, sz) in enumerate(zip(self.heads, self.sizes)):                        
            o = F.interpolate(mean, self.p_sz * sz, mode='bilinear')
            y = head(o, emb)
            layers.append(y)
        return U.join(layers)

class Decoder(nn.Module):
    def __init__(self, p_sz, e_sz, mid_sz, blocks):
        super().__init__()
        self.blocks = nn.ModuleList([
            MultiHead(p_sz, e_sz, mid_sz, sizes)
            for sizes  in blocks
        ])
    
    def forward(self, emb, mean):
        emb = emb.mean(dim=1, keepdim=True)        
        res = []
        for block in self.blocks:
            mean = block(emb, mean)
            res.append(mean)
        return res
        

p_sz = 8
e_sz = 512
mid_sz = 64
blocks = [
    (1, 1, 2, 2,),
    (3, 3, 4, 4,),
    (5, 6, 7, 8,),
    (10, 12, 14, 16,),
]
mean = torch.randn(1, 3, 256, 256).cuda()

decoder = Decoder(16, 512, 192, blocks).cuda()

decoder.load_state_dict(torch.load(f'./data/checkpoint/decoder512-{256}v.pth'))

optim = torch.optim.AdamW(decoder.parameters(), lr=0.0002)
optim.load_state_dict(torch.load(f'./data/checkpoint/optim/optim512-{256}v.pth'))

[e.shape for e in  decoder(torch.randn(7, 3, 512).cuda(), mean)]

[torch.Size([7, 3, 32, 32]),
 torch.Size([7, 3, 64, 64]),
 torch.Size([7, 3, 128, 128]),
 torch.Size([7, 3, 256, 256])]

In [3]:
device = torch.device('cuda')
subdivide = Subdivide().to(device)
#subdivide.smooth(patch_src, interpolate=True).shape
#patch_root = './data/fitted/64x64/'
patch_root = './data/fitted/512x512/'
emb_file = ['./data/face_emb/', './data/face_emb_hq/',]
n_embs = 3
#transform = lambda x: U.split(x[None], sizes=(16,))[0][0]

transform = lambda x: F.interpolate(x[None], 256)[0]
dataset = DecodeDataset(patch_root, emb_file, n_embs=n_embs, suffix='.pth', transform=transform)

patch_mean = dataset.patch_data.mean(dim=0, keepdim=True)
#patch_mean = patch_mean.div(patch_mean.abs().amax(dim=(1, 2, 3), keepdim=True))
#dataset = DecodeDataset(patch_root, emb_file, n_embs=n_embs, transform=transform)

for k, v  in dataset[0].items():
    print(k, v.shape if torch.is_tensor(v) else v)

emb_sizes = (512,) * 3
e_sz = 512
#decoder = Decoder(64, 256, 64, e_sz).to(device)

#optim = torch.optim.AdamW(decoder.parameters(), lr=0.0002)


#torch.save(patch_mean, './mean128.pt')
mean = dataset.patch_data.mean(dim=0, keepdim=True)
#patch_mean = patch_mean.div(patch_mean.abs().amax(dim=(1, 2, 3), keepdim=True))
#torch.save(patch_mean, 'mean256.pt')#
print(mean.shape)

emb, patch = [dataset[1][k][None].cuda() for  k in ('embedding', 'patch')]
emb.shape, patch.shape#, F.mse_loss(decoder(emb), patch)

patch torch.Size([3, 256, 256])
embedding torch.Size([3, 512])
idx 0
torch.Size([1, 3, 512, 512])


(torch.Size([1, 3, 512]), torch.Size([1, 3, 256, 256]))

In [4]:
loader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=0)
for g in optim.param_groups:
    #g['lr'] = 0.00075# 075
    print(g['lr'])

0.0001


In [7]:
keys = ('embedding', 'patch')# 'dino1', 'dino2')
#patch_mean =  patch_mean.to(device)
from src.shared.sharpen import Sharpen
sharpen = Sharpen(mask=('sharpen', '3x3_3'), padd=False).to(device)
#offsets = [F.interpolate(mean, sz).cuda() for sz in  (8, 16, 32, 24, 40, 48, 56, 64)]
#offsets = [F.interpolate(mean, sz).cuda() for sz in  [112, 256]] #[80, 112, 176, 256]]
mean = dataset.patch_data.mean(dim=0, keepdim=True)
mean = mean.div(mean.abs().amax(dim=(1, 2, 3), keepdim=True))
mean =  F.interpolate(mean, 32).cuda()

no_epochs = 1_001
err_lst0, err_lst1, err_lst2, err_lst3  = [], [], [], []
for epoch in range(no_epochs):
    for step, batch in enumerate(loader):
        #emb, patch_trg, mean, dino1, dino2 = (batch[k].cuda() for k in keys)
        emb, patch_trg = (batch[k].cuda() for k in keys)
        patch_trg = subdivide.smooth(patch_trg, interpolate=True)
        patch_trg = patch_trg.div(patch_trg.abs().amax(dim=(1, 2, 3), keepdim=True))
        
        patch_src_lst = decoder(emb, mean) #+ patch_mean
        errors =  [
            F.mse_loss(F.interpolate(patch_src, patch_trg.size(-1), mode='bilinear'), patch_trg)            
            for patch_src in patch_src_lst]
        for err_lst, err in zip([err_lst0, err_lst1, err_lst2, err_lst3], errors):
            err_lst.append(err.item())
        #diff = (patch_src - patch_trg).abs()#.sum(dim=1, keepdim=True)
        #mse_err = diff.mean() +  F.max_pool2d(diff, 16).mean()
        #mse_err = F.mse_loss(patch_src, patch_trg)
        
        error = sum(errors) #+ 0.01 * F.l1_loss(patch_src, patch_trg)
        
        optim.zero_grad()
        error.backward()
        optim.step()
        #err_lst.append(error.item())
        if step % 100 == 0:
            temperature = GPUtil.getGPUs()[0].temperature
            if epoch % 1 == 0:
                print(str(epoch).zfill(4), str(step).zfill(4), 
                      f'{torch.tensor(err_lst0).mean().item():.8f}',
                      f'{torch.tensor(err_lst1).mean().item():.8f}',
                      f'{torch.tensor(err_lst2).mean().item():.8f}',
                      f'{torch.tensor(err_lst3).mean().item():.8f}',
                      #f'{torch.tensor(sharpen_lst).mean().item():.8f}',
                      temperature)
                err_lst0, err_lst1, err_lst2, err_lst3  = [], [], [], []                             
                
            if  temperature > 92:
                while temperature > 68:
                    print(f'GPU:{temperature}')
                    time.sleep(30)
                    temperature = GPUtil.getGPUs()[0].temperature
    if epoch % 3 ==0:      
        size = patch_src_lst[-1].size(-1)
        torch.save(decoder.state_dict(), f'./data/checkpoint/decoder512-{size}v.pth')
        torch.save(optim.state_dict(), f'./data/checkpoint/optim/optim512-{size}v.pth')
        for i, (pc, ps, pt) in enumerate(zip(patch_src_lst[0], patch_src_lst[-1], patch_trg)):
            if i < 3:
                U.export_stl(pc, f'{i}crc')
                U.export_stl(ps, f'{i}src')
                U.export_stl(pt, f'{i}trg')

size = patch_src_lst[-1].size(-1)
torch.save(decoder.state_dict(), f'./data/checkpoint/decoder512-{size}v.pth')
torch.save(optim.state_dict(), f'./data/checkpoint/optim/optim512-{size}v.pth')
# 0059 0000 0.00003770 0.00003576 0.00004650 0.00004472 93.0

0000 0000 0.00005880 0.00004860 0.00004861 0.00005669 72.0
0000 0100 0.00044868 0.00060002 0.00338828 0.00734876 82.0
0000 0200 0.00053034 0.00112699 0.00194456 0.01270921 85.0
0000 0300 0.00036057 0.00050466 0.00054381 0.00056111 85.0
0001 0000 0.00023965 0.00025564 0.00025672 0.00028042 86.0
0001 0100 0.00019688 0.00020155 0.00020542 0.00022304 86.0
0001 0200 0.00015525 0.00015698 0.00016126 0.00017482 87.0
0001 0300 0.00014462 0.00014374 0.00014876 0.00015991 88.0
0002 0000 0.00013014 0.00012886 0.00013418 0.00014357 89.0
0002 0100 0.00011856 0.00011589 0.00012087 0.00012894 88.0
0002 0200 0.00010868 0.00010460 0.00010955 0.00011682 89.0
0002 0300 0.00010181 0.00009730 0.00010215 0.00010892 90.0
0003 0000 0.00009748 0.00009209 0.00009722 0.00010317 91.0
0003 0100 0.00009199 0.00008553 0.00008999 0.00009551 91.0
0003 0200 0.00008927 0.00008281 0.00008716 0.00009206 91.0
0003 0300 0.00008636 0.00007942 0.00008342 0.00008821 91.0
0004 0000 0.00008460 0.00007688 0.00008065 0.00008488 91

KeyboardInterrupt: 

In [8]:
errors

[tensor(    0.00006, device='cuda:0', grad_fn=<MseLossBackward0>),
 tensor(    0.00004, device='cuda:0', grad_fn=<MseLossBackward0>),
 tensor(    0.00005, device='cuda:0', grad_fn=<MseLossBackward0>),
 tensor(    0.00005, device='cuda:0', grad_fn=<MseLossBackward0>)]

In [15]:
size = patch_trg.size(-1)
torch.save(decoder.state_dict(), f'./data/checkpoint/decoder512-{size}v.pth')
torch.save(optim.state_dict(), f'./data/checkpoint/optim/optim512-{size}v.pth')

In [9]:
torch.save(decoder.state_dict(), f'decoder512-{patch_trg.size(-1)}_{8}.pth')

In [14]:
for i, (pc, ps, pt) in enumerate(zip(patch_src_lst[0], patch_src_lst[-1], patch_trg)):
    if i < 3:
        U.export_stl(pc, f'{i}crc')
        U.export_stl(ps, f'{i}src')
        U.export_stl(pt, f'{i}trg')

In [ ]:
print(str(epoch).zfill(4), str(step).zfill(4), 
                      f'{torch.tensor(err_lst).mean().item():.8f}', temperature)

In [ ]:
loader = DataLoader(dataset, batch_size=8, shuffle=False, num_workers=0)
decoded512_32 = torch.zeros(len(dataset), 3, 32, 32)
orig_512_32 = torch.zeros(len(dataset), 3, 32, 32)
device = torch.device('cuda')

bsz = 8
keys = ('embedding', 'patch')
for i, batch in enumerate(loader):
    emb, orig = (batch[k].to(device) for k in keys)    
    with torch.no_grad():
        coarse = decoder(emb)        
    decoded512_32[i*bsz: (i+1) * bsz] = coarse
    orig_512_32[i*bsz: (i+1) * bsz] = orig

torch.save(decoded512_32, './data/decoded512_32j.pt')

In [ ]:
for i in [100, 700, 2300]:
    U.export_stl(orig_512_32[i], f'{i}org')
    U.export_stl(decoded512_32[i], f'{i}src')

In [ ]:
dataset.patch_data.shape